# Big Data Project H600 / L-Group
***
## Real world Data Exploration, Integration, Cleasning, Transformation and Analysis

Data:
The New York City Taxi and Limousine Commission (or TLC for short) has been publishing
records about taxi trips in New York since 2009. 

The TLC trip dataset actually consists of 4 sub-datasets:

    1.Yellow taxi records are records that record trip information of New York's famous yellow taxi cars

    2.Green taxi records are records that record trip information by so-called 'boro' taxis, a newer service introduced in August of 2013 to improve taxi service and availability in the boroughs. 

    3.FHV records (short for 'For Hire Vehicles') record information from services that offered for-hire vehicles (such as Uber, Lyft, Via, and Juno), but also luxury limousine bases.

    4.High volume FHV (FHVHV for short) are FHV records offered by services that make more than 10,000 trips per day

In [33]:
conda install -c conda-forge matplotlib

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [1]:
import os 
os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=3g  pyspark-shell"
from pyspark.sql import SparkSession
try: 
    spark
    print("Spark application already started. Terminating existing application and starting new one")
    spark.stop()
except: 
    pass
# Create a new spark session (note, the * indicates to use all available CPU cores)
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("H600 L-Group") \
    .getOrCreate()
#When dealing with RDDs, we work the sparkContext object. See https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext
sc=spark.sparkContext
#in local mode, you will be able to access the Spark GUI at http://localhost:4040

## 1: Files & Data Exploration 
### 1.1 Count of files per type

In [106]:
print ('count of green tripdata files:')
!find /home/bigdata/Desktop/Project/Data/green_tripdata_*.csv -type f | wc -l 
print ('count of yellow tripdata files:')
!find /home/bigdata/Desktop/Project/Data/yellow_tripdata_*.csv -type f | wc -l
print ('count of fhv tripdata files:')
!find /home/bigdata/Desktop/Project/Data/fhv_tripdata_*.csv -type f | wc -l 
print ('count of fhvhv tripdata files:')
!find /home/bigdata/Desktop/Project/Data/fhvhv_tripdata_*.csv -type f | wc -l
print ('count of all tripdata files:')
!find /home/bigdata/Desktop/Project/Data/*.csv -type f | wc -l 

count of green tripdata files:
76
count of yellow tripdata files:
131
count of fhv tripdata files:
64
count of fhvhv tripdata files:
10
count of all tripdata files:
281


### 1.2 Files size (bytes) and metrics

In [202]:
import os
import glob
import pandas as pd
import math
import numpy as np


os.chdir("/home/bigdata/Desktop/Project/Data/")
taxi_brand="fhv"
list_files = {}
list_files[taxi_brand] = []
nb_files = 0
size_b = []
# List the file from the same taxi company brand 
for file in glob.glob("%s*.csv" %(taxi_brand)):
    nb_files = nb_files+1
    # Save in list the file name
    list_files[taxi_brand].append(file)
    size_b.append(os.path.getsize(file))

print("For the taxi brand called %s there are %i files." %(taxi_brand, nb_files))
# Get size basic stat
print("Here are some metrics based on their size in bytes:")
print("-min value           : ", np.min(size_b))
print("-max value           : ", np.max(size_b))
print("-mean value          : ", np.mean(size_b))
print("-25 percentile value : ", np.quantile(size_b, .25)) 
print("-50 percentile value : ", np.quantile(size_b, .50)) 
print("-75 percentile value : ", np.quantile(size_b, .75)) 
print("-90 percentile value : ", np.quantile(size_b, .90)) 


For the taxi brand called fhv there are 74 files.
-min value           :  52060
-max value           :  3339455
-mean value          :  1263540.6621621621
-25 percentile value :  239082.25
-50 percentile value :  725289.0
-75 percentile value :  2545631.75
-90 percentile value :  3003289.6


### 1.3 Files size (rows) and metrics

In [206]:
names={}
size_r=[]

for fn in glob.glob("%s*.csv" %(taxi_brand)):
    with open(fn) as f:
        names[fn]=sum(1 for line in f if line.strip())     
        
    # Save in list files sizes in rows
    size_r=list(names.values())

print("For the taxi brand called %s there are still %i files." %(taxi_brand, nb_files))
# Get size basic stat
print("Here are some metrics based on their size in rows count:")
print("-min value           : ", np.min(size_r))
print("-max value           : ", np.max(size_r))
print("-mean value          : ", np.mean(size_r))
print("-25 percentile value : ", np.quantile(size_r, .25)) 
print("-50 percentile value : ", np.quantile(size_r, .50)) 
print("-75 percentile value : ", np.quantile(size_r, .75)) 
print("-90 percentile value : ", np.quantile(size_r, .90)) 

For the taxi brand called fhv there are still 74 files.
Here are some metrics based on their size in rows count:
-min value           :  960
-max value           :  47704
-mean value          :  23128.391891891893
-25 percentile value :  7975.25
-50 percentile value :  22391.5
-75 percentile value :  39336.5
-90 percentile value :  44120.3


In [10]:
# Load the contents of a file into an RDD. Note - when run on the cluster this load from HDFS (inside /user/$USER/)
# if you really want to load from HDFS, you can also put the full HDFS url, e.g.
# hdfs://public00:8020/user/<your_user_id_here>/data/books/pg20417.txt
fileName = 'Data/green_tripdata_2020-01.csv'
TaxiRDD = sc.textFile(fileName)

In [11]:
TaxiRDD.take(5)

['VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge',
 ',2020-01-27 06:40:00,2020-01-27 07:25:00,,,159,61,,19.84,40.95,2.75,0,0,6.12,,0.3,50.12,,,',
 '2,2020-01-18 01:01:42,2020-01-18 01:04:57,N,1,7,146,1,.56,4.5,0.5,0.5,1.45,0,,0.3,7.25,1,1,0',
 '2,2020-01-15 13:52:01,2020-01-15 14:13:09,N,1,134,77,1,5.87,22,0,0.5,0,0,,0.3,22.8,1,1,0',
 '2,2020-01-30 23:49:37,2020-01-31 00:12:17,N,1,42,143,1,5.40,20,0.5,0.5,4.81,0,,0.3,28.86,1,1,2.75']

In [13]:
tupleRDD = TaxiRDD.map(lambda line: line.split())
tupleRDD.take(3)

[['VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge'],
 [',2020-01-27',
  '06:40:00,2020-01-27',
  '07:25:00,,,159,61,,19.84,40.95,2.75,0,0,6.12,,0.3,50.12,,,'],
 ['2,2020-01-18',
  '01:01:42,2020-01-18',
  '01:04:57,N,1,7,146,1,.56,4.5,0.5,0.5,1.45,0,,0.3,7.25,1,1,0']]

In [14]:
wordsRDD = TaxiRDD.flatMap(lambda line: line.split())
wordsRDD.take(5)

['VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge',
 ',2020-01-27',
 '06:40:00,2020-01-27',
 '07:25:00,,,159,61,,19.84,40.95,2.75,0,0,6.12,,0.3,50.12,,,',
 '2,2020-01-18']

In [15]:
wordsRDD.count()

2686

In [16]:
wordsRDD.map(lambda x: 1).reduce(lambda a,b: a+b)

2686

### Do a word count on all files

In [17]:
allTaxiRDD = sc.textFile('Data/*.csv')

In [19]:
allTaxiRDD.count()

5036382

In [20]:
allTaxiRDD.count()

5036382

### Yellow taxi records